In [12]:
print("ok")

ok


In [92]:
import os
os.chdir("/Users/anilkatwal/Desktop/Medical_chatbot/Medical_chatbot")



In [93]:
os.getcwd()


'/Users/anilkatwal/Desktop/Medical_chatbot/Medical_chatbot'

In [94]:
%pwd

'/Users/anilkatwal/Desktop/Medical_chatbot/Medical_chatbot'

In [99]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [101]:
from langchain_community.document_loaders import DirectoryLoader, PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

def load_pdf_file(data: str):
    print("Starting to load PDF files...")

    # Use PyMuPDFLoader instead of PyPDFLoader for faster loading
    loader = DirectoryLoader(
        data,
        glob="**/*.pdf",
        loader_cls=PyMuPDFLoader  # ← updated here
    )
    documents = loader.load()
    print(f"Loaded {len(documents)} documents")

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200
    )
    texts = text_splitter.split_documents(documents)
    print(f"Split into {len(texts)} chunks")

    return texts


In [102]:
extracted_data=load_pdf_file(data='Data/')

Starting to load PDF files...
Loaded 4505 documents
Split into 23586 chunks


In [17]:
#extracted_data

In [105]:
from langchain_community.embeddings import HuggingFaceEmbeddings

def download_hugging_face_embedding():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2')
    return embeddings

embeddings = download_hugging_face_embedding()


In [138]:
#! pip install "pinecone-client[grpc]"

#print(embedding)

In [106]:
embeddings = download_hugging_face_embedding()


In [126]:
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [127]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')

In [128]:
import os
print(os.getenv("PINECONE_API_KEY"))


pcsk_4AgfgL_GaSCoDREkHM6gj7iwzmugjLRtHZYMFwLNbrHXEM7ccUrQgwFA1uGkhSYCk98Lih


In [129]:
from dotenv import load_dotenv
import os

load_dotenv()

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_ENV = os.getenv("PINECONE_ENV")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


In [130]:
from pinecone import Pinecone

pc = Pinecone(api_key=PINECONE_API_KEY)


In [135]:
import os
os.environ["PINECONE_API_KEY"]=PINECONE_API_KEY

# Embed each chunk and upsert the embedding into your Pinecone index

In [140]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import DirectoryLoader, PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone
import os

# Load Pinecone API key securely
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv("PINECONE_API_KEY")
if not api_key:
    raise EnvironmentError("PINECONE_API_KEY is not set.")
os.environ["PINECONE_API_KEY"] = api_key

# Load and split PDFs
def load_pdf_file(data: str):
    loader = DirectoryLoader(data, glob="**/*.pdf", loader_cls=PyMuPDFLoader)
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    texts = text_splitter.split_documents(documents)

    # Filter empty chunks and large ones
    filtered = []
    for i, doc in enumerate(texts):
        size = len(doc.page_content.encode('utf-8'))
        if 0 < size < 4_000_000:
            # Optionally simplify metadata
            doc.metadata = {
                "source": doc.metadata.get("source", "")[:100],
                "page": doc.metadata.get("page", -1)
            }
            filtered.append(doc)
        elif size >= 4_000_000:
            print(f"⚠️ Skipping large chunk {i}: {size} bytes")
    return filtered

# Define embedding function
def get_embedding_model():
    return HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2')

# Load PDFs and get text chunks
data_directory = "./Data/"
text_chunks = load_pdf_file(data_directory)

# Initialize embedding model
embeddings = get_embedding_model()

# Verify embedding dimension
sample_embedding = embeddings.embed_query("test")
embedding_dim = len(sample_embedding)
print(f"Embedding dimension: {embedding_dim}")

# Initialize Pinecone
pc = Pinecone(api_key=api_key)
index_name = "medicalchatbot"

# Create or get existing index
if index_name in pc.list_indexes().names():
    index = pc.Index(index_name)
    stats = index.describe_index_stats()
    if stats['dimension'] != embedding_dim:
        raise ValueError(f"Existing index dimension {stats['dimension']} does not match embedding dimension {embedding_dim}")
else:
    pc.create_index(name=index_name, dimension=embedding_dim, metric="cosine")

# Batch upload documents to Pinecone
batch_size = 50
for i in range(0, len(text_chunks), batch_size):
    batch = text_chunks[i:i + batch_size]
    print(f"📤 Uploading batch {i // batch_size + 1} with {len(batch)} documents...")
    PineconeVectorStore.from_documents(
        documents=batch,
        index_name=index_name,
        embedding=embeddings
    )

# Test a sample query
query = "What medical topics are covered in the PDFs?"
docsearch = PineconeVectorStore.from_existing_index(index_name=index_name, embedding=embeddings)
results = docsearch.similarity_search(query, k=1)
print("🔍 Top result:\n", results[0].page_content)


Embedding dimension: 768
📤 Uploading batch 1 with 50 documents...
📤 Uploading batch 2 with 50 documents...
📤 Uploading batch 3 with 50 documents...
📤 Uploading batch 4 with 50 documents...
📤 Uploading batch 5 with 50 documents...
📤 Uploading batch 6 with 50 documents...
📤 Uploading batch 7 with 50 documents...
📤 Uploading batch 8 with 50 documents...
📤 Uploading batch 9 with 50 documents...
📤 Uploading batch 10 with 50 documents...
📤 Uploading batch 11 with 50 documents...
📤 Uploading batch 12 with 50 documents...
📤 Uploading batch 13 with 50 documents...
📤 Uploading batch 14 with 50 documents...
📤 Uploading batch 15 with 50 documents...
📤 Uploading batch 16 with 50 documents...
📤 Uploading batch 17 with 50 documents...
📤 Uploading batch 18 with 50 documents...
📤 Uploading batch 19 with 50 documents...
📤 Uploading batch 20 with 50 documents...
📤 Uploading batch 21 with 50 documents...
📤 Uploading batch 22 with 50 documents...
📤 Uploading batch 23 with 50 documents...
📤 Uploading batch 

In [142]:
#load Existing Index
from langchain_pinecone import PineconeVectorStore
docsearch=PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
    
)


In [143]:
docsearch

In [182]:
retriever=docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [196]:
retrieved_docs=retriver.invoke("what is Acne?")

In [197]:
retrieved_docs

[Document(id='8c379ec0-273d-4acd-9083-329cb6392d00', metadata={'page': 54.0, 'source': 'Data/The-Gale-Encyclopedia-of-Medicine-3rd-Edition-staibabussalamsula.ac_.id_.pdf'}, page_content='Acne is a common skin disease characterized by\npimples on the face, chest, and back. It occurs when\nthe pores of the skin become clogged with oil, dead\nskin cells, and bacteria.\nDescription\nAcne vulgaris, the medical term for common acne,\nis the most common skin disease. It affects nearly 17\nmillion people in the United States. While acne can\narise at any age, it usually begins at puberty and wor-\nsens during adolescence. Nearly 85% of people\ndevelop acne at some time between the ages of 12-25'),
 Document(id='4d245419-92a2-4541-9c65-a57a201e132e', metadata={'page': 3353.0, 'source': 'Data/The-Gale-Encyclopedia-of-Medicine-3rd-Edition-staibabussalamsula.ac_.id_.pdf'}, page_content='KEY TERMS\nAcne—A chronic inflammation of the sebaceous\nglands that manifests as blackheads, whiteheads,\nand/o

In [174]:
# ! pip install langchain-xai

In [160]:
import os
os.environ["XAI_API_KEY"]=XAI_API_KEY

In [175]:
from langchain_xai import ChatXAI

llm = ChatXAI(
    model="grok-beta",
    temperature=0.4,
    max_tokens=500,
    api_key=XAI_API_KEY
)

In [195]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt=(
    "you are an assistant for question_answering tasks."
    "Use the following pieces of retrived context to answer"
    "the question. If you don't know the answer say that you "
    "don't know. use the three sentences maximum and keep the"
    "answer concise."
    "\n\n"
    "{context}"
)

In [198]:
prompt= ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human","{input}"),
    ]
)

In [199]:
question_answer_chain=create_stuff_documents_chain(llm, prompt)
rag_chain=create_retrieval_chain(retriever, question_answer_chain)

In [202]:
response=rag_chain.invoke({"input":"what is Ache?"})
print(response["answer"])